# 01 - Génération de données fiscales synthétiques (France)
Objectif : créer 50 documents contenant de la PII réaliste pour tester la détection + anonymisation.

In [ ]:
import snowflake.snowpark as snp
import pandas as pd
from faker import Faker
import random
from datetime import datetime, date

# Initialisation de la session Snowpark et de Faker
session = get_active_session()   
fake = Faker('fr_FR') # Locale française pour des données réalistes 
print(" Setup complet, prêt à générer 50 documents")

Génération de 50 documents synthétiques


In [ ]:
# Générer 50 documents synthétiques
documents = []

for i in range(50):
    # Identifiants et infos perso
    nom = fake.last_name()
    prenom = fake.first_name()
    date_naissance = fake.date_of_birth(minimum_age=18, maximum_age=80)
    age = date.today().year - date_naissance.year

     # Entreprise & identifiants légaux
    entreprise = fake.company()
    siren = fake.siren()  
    siret = fake.siret()  

    # Coordonnées & contact
    adresse = fake.address().replace("\n", ", ")
    code_postal = fake.postcode()  
    ville = fake.city()
    email = fake.email()
    telephone = fake.phone_number()  

    # Données bancaires / identifiants
    iban = fake.iban() 

    # Données financières
    revenu = random.randint(20_000, 150_000)
    impot = int(revenu * random.uniform(0.15, 0.35))
    autres_revenus = random.randint(1_000, 30_000)

    texte = f"""
DÉCLARATION FISCALE FRANÇAISE

Identité du déclarant
Nom : {nom}
Prénom : {prenom}
Date de naissance : {date_naissance.strftime('%d/%m/%Y')}
Âge : {age} ans

Coordonnées
Adresse : {adresse}
Téléphone : {telephone}
Email : {email}

Informations professionnelles
Entreprise : {entreprise}
SIREN : {siren}
SIRET : {siret}
Lieu de travail : {ville}

Informations bancaires
IBAN : {iban}

Données financières
Revenu brut annuel : {revenu:,}€
Autres revenus déclarés : {autres_revenus:,}€
Impôts estimés : {impot:,}€

Ce document contient des données personnelles sensibles (RGPD) simulées à des fins de test.
    """.strip()

    documents.append({
        'DOC_ID': 1000 + i,
        'FILENAME': f'synthetic_{i+1:03d}.txt',
        'DOCUMENT_TEXT': texte,
        'DOCUMENT_TYPE': 'synthetic_fiscal',
        'DOC_LENGTH': len(texte),
    })

print(f"{len(documents)} documents générés")

# Aperçu des 1ers documents
df_docs = pd.DataFrame(documents)
df_docs.head()

In [ ]:
doc_id = 1000  # ou un autre ID
doc = next(d for d in documents if d["DOC_ID"] == doc_id)

print(doc["DOCUMENT_TEXT"])

# Insérer les documents générés dans la table

In [ ]:
# Charger les documents synthétiques dans Snowflake
df_docs = pd.DataFrame(documents)
sdf_docs = session.create_dataframe(df_docs)

# On écrase ou on remplit la table RAW_DATA.FISCAL_DOCUMENTS_RAW
sdf_docs.write.mode("overwrite").save_as_table("RAW_DATA.FISCAL_DOCUMENTS_RAW")

print(f"{len(documents)} documents chargés dans Snowflake")

# Vérifier
result = session.sql("SELECT COUNT(*) as count FROM RAW_DATA.FISCAL_DOCUMENTS_RAW").collect()
print(f"Total en Snowflake: {result[0]['COUNT']}")


In [ ]:
USE DATABASE PII_ANONYMIZATION;
USE SCHEMA RAW_DATA;

SELECT CURRENT_DATABASE() AS DB, CURRENT_SCHEMA() AS SCHEMA;

SHOW TABLES IN SCHEMA RAW_DATA;

SELECT COUNT(*) FROM RAW_DATA.FISCAL_DOCUMENTS_RAW;


In [ ]:
SELECT DOC_ID, DOCUMENT_TEXT
FROM RAW_DATA.FISCAL_DOCUMENTS_RAW
WHERE DOC_ID = 1000;  

In [ ]:
SELECT DOC_ID, DOCUMENT_TEXT
FROM RAW_DATA.FISCAL_DOCUMENTS_RAW